In [ ]:
import os
import sys

sys.path.append("..")

from lib.eval.silk_runner import SilkRunner
from lib.eval.eval import Eval
from lib.dataset.mapfree import MapFreeDataset
from yacs.config import CfgNode as CN
from torch.utils.data import DataLoader 

In [ ]:
def init_config(dataset_config: str) -> CN:
    node = CN()
    node.set_new_allowed(True)
    node.merge_from_file(dataset_config)
    node.DEBUG = False

    # explicitely setting to None because if loading from yaml it's a string
    node.DATASET.SCENES = None
    node.DATASET.AUGMENTATION_TYPE = None
    return node

In [ ]:
silk_config = "../config/silk/silk_config.yaml"
checkpoint = "../../silk/assets/models/silk/coco-rgb-aug.ckpt"
data_dir = "../data"
dataset_config = "../config/datasets/mapfree.yaml"
dataset_split = "val"

paths = [silk_config, checkpoint, data_dir, dataset_config]
exists = [os.path.exists(p) for p in paths]
if sum(exists) == len(paths):
    print("All paths exist :)")

In [ ]:
mapfree_config = init_config(dataset_config)
mapfree_config.DATASET.DATA_ROOT = "../" + mapfree_config.DATASET.DATA_ROOT
mapfree_config.DATASET.DEPTH_ROOT = "../../mickey_depths"
dataset = MapFreeDataset(mapfree_config, dataset_split)
loader = DataLoader(dataset, batch_size=4)

In [ ]:
runner = SilkRunner(silk_config, checkpoint)
estimated_poses = runner.run(loader)

In [ ]:
results = Eval.fromMapFree(estimated_poses, dataset)

In [ ]:
eval_results = results

In [ ]:
results = eval_results.results

In [ ]:
results.keys()

In [ ]:
import statistics

statistics.mean(results['trans_err'])

In [ ]:
statistics.median(results['trans_err'])

In [ ]:
for k in results.keys():
    print(f"{k} median: {statistics.median(results[k])}")
    print(f"{k} mean: {statistics.mean(results[k])}")